# Setting up

## load modules

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import numpy as np
from tqdm import tqdm
from os import listdir
from pathlib import Path
import cv2
import matplotlib
import shutil

## set path and parameters

In [ ]:
# Define file path here. 
dpath = "FILE_PATH_HERE"
dpath = os.path.abspath(dpath)

## Look for striped frames again

In [ ]:
def replace_LEDoff_frames(fpath, replacement_frame_number=3):
    
    # Get 
    folder = os.path.join(os.path.split(fpath)[0], 'originals')
    if not os.path.exists(folder):
        os.mkdir(folder)
    fname = os.path.split(fpath)[1]
    move_fpath = os.path.join(folder, fname)
    
    # Move the original file. 
    if not os.path.exists(move_fpath):
        shutil.move(fpath, move_fpath)
        print(f'Moved {fpath} to {move_fpath}')
    else:
        raise FileExistsError('The folder is already storing an original 0.avi. Aborting to prevent overwrite.')
    
    # Read the original file.
    compressionCodec = "FFV1"
    codec = cv2.VideoWriter.fourcc(*compressionCodec)
    cap = cv2.VideoCapture(move_fpath)
    rows, cols = int(cap.get(4)), int(cap.get(3))
        
    # Get replacement frame.
    cap.set(cv2.CAP_PROP_POS_FRAMES, replacement_frame_number)
    _, replacement_frame = cap.read()
    replacement_frame = replacement_frame[:,:,1]
    
    #Return to the first frame.
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    writeFile = cv2.VideoWriter(fpath, codec, 60, (cols, rows), isColor=False)
    # Rewrite video file. 
    print(f'Writing new {fpath}')
    for frame_number in tqdm(range(int(cap.get(7)))):
        ret, frame = cap.read()

        if ret:
            if frame_number < replacement_frame_number:
                writeFile.write(np.uint8(replacement_frame))
            else:
                writeFile.write(np.uint8(frame[:,:,1]))
            
        else:
            break
        
    writeFile.release()
    cap.release()
    cv2.destroyAllWindows()
    

In [ ]:
replace_LEDoff_frames(os.path.join(dpath,'0.avi'), 3)

There should now be a folder called "final" that should contain the final products. Rename the files you recently moved from "repaired" to something else (i.e., "0_striped.avi") and replace them with the "final" videos. 